In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from ultralytics import YOLO
from collections import Counter
import torch
from pathlib import Path

In [ ]:
# Charger l'image de l'abeille et le masque
image = cv2.imread(r'C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\\train\\Test 23-05-2024\\DATA_TEST\\300.jpg')
mask = cv2.imread(r'C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\\train\\Test 23-05-2024\\DATA_TEST\\mask\\binary_300.tif', cv2.IMREAD_GRAYSCALE)

# Identifier la zone blanche de l'abeille dans le masque
contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
largest_contour = max(contours, key=cv2.contourArea)
x, y, w, h = cv2.boundingRect(largest_contour)

# Déterminer les paramètres de zoom
zoom_factor = 1.05  # Choisissez le facteur de zoom selon vos préférences
zoomed_x = max(0, x - w * (zoom_factor - 1) / 2)
zoomed_y = max(0, y - h * (zoom_factor - 1) / 2)
zoomed_w = min(image.shape[1], w * zoom_factor)
zoomed_h = min(image.shape[0], h * zoom_factor)

# Appliquer les paramètres de zoom à l'image
zoomed_image = image[int(zoomed_y):int(zoomed_y + zoomed_h), int(zoomed_x):int(zoomed_x + zoomed_w)]

# Enlever le fond de l'image en utilisant le masque
masked_zoomed_image = cv2.bitwise_and(zoomed_image, zoomed_image, mask=mask[int(zoomed_y):int(zoomed_y + zoomed_h), int(zoomed_x):int(zoomed_x + zoomed_w)])

# Enregistrer l'image zoomée sans fond
cv2.imwrite('IMG_ZX_1.jpg', masked_zoomed_image)

# Afficher les quatre images
plt.figure(figsize=(12, 8))

plt.subplot(2, 2, 1)
plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
plt.title('Image Originale')
plt.axis('off')

plt.subplot(2, 2, 2)
plt.imshow(cv2.cvtColor(zoomed_image, cv2.COLOR_BGR2RGB))
plt.title('Image Zoomée')
plt.axis('off')

plt.subplot(2, 2, 3)
plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
plt.title('Image Originale sans Fond')
plt.axis('off')

plt.subplot(2, 2, 4)
plt.imshow(cv2.cvtColor(masked_zoomed_image, cv2.COLOR_BGR2RGB))
plt.title('Image Zoomée sans Fond')
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Créer un dossier pour les images zoomées sans fond
output_folder = 'DATA_TEST_TRANSFORM'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
remaining_images = 0

# Parcourir les images d'abeille
for index, file in enumerate(os.listdir(r'C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\\train\\Test 23-05-2024\\DATA_TEST')):
    if file.endswith('.JPG'):  # Assurez-vous de sélectionner uniquement les fichiers JPG
        # Charger l'image de l'abeille
        image_path = os.path.join(r'C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\\train\\Test 23-05-2024\\DATA_TEST', file)
        image = cv2.imread(image_path)
        
        # Trouver le numéro de l'image pour correspondre au masque
        image_number = int(os.path.splitext(file)[0])
        
        # Charger le masque correspondant
        mask_path = os.path.join(r'C:\\Users\\marti\\OneDrive\\Martin OneDrive\\ISEP\\A2\\Semestre 2\\Ai and optimisation\\Projet\\train\\Test 23-05-2024\\DATA_TEST\\masks', f'binary_{image_number}.tif')
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        
        # Identifier la zone blanche de l'abeille dans le masque
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)
        
        # Déterminer les paramètres de zoom
        zoom_factor = 1.05  # Choisissez le facteur de zoom selon vos préférences
        zoomed_x = max(0, x - w * (zoom_factor - 1) / 2)
        zoomed_y = max(0, y - h * (zoom_factor - 1) / 2)
        zoomed_w = min(image.shape[1], w * zoom_factor)
        zoomed_h = min(image.shape[0], h * zoom_factor)
        
        # Appliquer les paramètres de zoom à l'image
        zoomed_image = image[int(zoomed_y):int(zoomed_y + zoomed_h), int(zoomed_x):int(zoomed_x + zoomed_w)]
        
        # Enlever le fond de l'image en utilisant le masque
       # masked_zoomed_image = cv2.bitwise_and(zoomed_image, zoomed_image, mask=mask[int(zoomed_y):int(zoomed_y + zoomed_h), int(zoomed_x):int(zoomed_x + zoomed_w)])
        
        # Enregistrer l'image zoomée sans fond
        output_path = os.path.join(output_folder, f'{image_number}.jpg')
        if os.path.exists(output_path):
            os.remove(output_path)  # Supprimer l'ancienne image si elle existe
            
        cv2.imwrite(output_path, zoomed_image)

        # Afficher le compte à rebours
        remaining_images = remaining_images + 1
        print(f'{remaining_images} images ont été enregistrées. {image_number}')
        
print('Toutes les images dans le dossier ont été enregitrées')

In [ ]:
results_path = './runs/classify/train7/results.csv'

results = pd.read_csv(results_path)

results

In [ ]:
model = YOLO('./runs/classify/train7/weights/last.pt')  # load a custom model

In [ ]:
# Dossier contenant les images
image_folder = './DATA_TEST/'

# Fichier CSV de sortie
output_csv = 'predictions.csv'

# Liste pour stocker les résultats
results_list = []

In [2]:
# Parcourir toutes les images du dossier
for image_name in os.listdir(image_folder):
    if image_name.endswith(('.jpg')):  # Filtrer les fichiers d'image
        image_path = os.path.join(image_folder, image_name)
        
        # Faire la prédiction
        results = model(image_path)  # Prédire sur l'image
        
        # Extraire les noms des classes et les probabilités
        names_dict = results[0].names
        probs = results[0].probs.data.tolist()
        
        # Trouver la classe avec la probabilité la plus élevée
        max_index = np.argmax(probs)
        predicted_class = names_dict[max_index]
        confidence = probs[max_index]
        
        # Ajouter les résultats à la liste
        results_list.append([image_name, predicted_class, confidence])
        
# Écrire les résultats dans un fichier CSV
with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image Name', 'Predicted Class', 'Confidence'])
    writer.writerows(results_list)

print(f"Les résultats des prédictions ont été enregistrés dans {output_csv}.")

NameError: name 'image_folder' is not defined

In [ ]:
# Lire les données du fichier CSV
predictions = []
with open(output_csv, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        predictions.append(row['Predicted Class'])

# Compter le nombre de prédictions pour chaque classe
prediction_counts = Counter(predictions)

# Trier les classes par ordre décroissant de fréquence
sorted_counts = prediction_counts.most_common()

# Séparer les noms des classes et les compteurs pour l'affichage
classes, counts = zip(*sorted_counts)

# Générer l'histogramme
plt.figure(figsize=(10, 6))
plt.bar(classes, counts, color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Nombre de Prédictions')
plt.title('Nombre de Prédictions pour chaque Classe')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Lire les données du fichier CSV
predictions = defaultdict(list)
with open(output_csv, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        class_name = row['Predicted Class']
        confidence = float(row['Confidence'])
        predictions[class_name].append(confidence)

# Calculer le pourcentage moyen de confiance pour chaque classe
average_confidences = {class_name: sum(confidences) / len(confidences)
                       for class_name, confidences in predictions.items()}

# Trier les classes par ordre décroissant de confiance moyenne
sorted_avg_confidences = sorted(average_confidences.items(), key=lambda x: x[1], reverse=True)

# Séparer les noms des classes et les moyennes pour l'affichage
classes, avg_confidences = zip(*sorted_avg_confidences)

# Générer l'histogramme
plt.figure(figsize=(10, 6))
plt.bar(classes, avg_confidences, color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Confiance Moyenne')
plt.title('Confiance Moyenne des Prédictions pour chaque Classe')
plt.xticks(rotation=45)
plt.ylim(0, 1)  # Les pourcentages de confiance sont entre 0 et 1
plt.show()